In [8]:
'''
匯入套件
'''

# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

#美麗湯
from bs4 import BeautifulSoup as bs

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 子處理程序，用來取代 os.system 的功能
import subprocess

#正規表達式
import re

# 下載檔案的工具
import wget

import pandas as pd


# 由於klook 會擋 嘗試更換user-agent
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文
my_options.add_argument(f"user-agent={user_agent}")
# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options
)

# 放置爬取的資料
listData = []

In [9]:
# 先嘗試爬取第一頁

# 前往klook 故宮門票頁面
url = 'https://www.klook.com/zh-TW/activity/10136-national-palace-museum-ticket-package-taipei/'
driver.get(url)



# 等待元素載入完畢
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located( 
        (By.CSS_SELECTOR, 'p.content') 
    )
)

sleep(3)

# 取得網頁原始碼
html = driver.page_source
#print(html)
    
# 用美麗湯重新編譯
soup = bs(html, 'lxml')  
#print(soup)
    
# 清空列表
listData.clear()
    
# 取得所有評論
content = soup.select("p.content")
# 取得所有評論時間
date = soup.select("p.review-user-time") 
    
for time,comment in zip(date,content):
        
    # 調整時間格式
    regex = r'/D'
    format_time = re.sub(regex,'',time)

      
        
    listData.append({
        'sources':"Klook",
        'title': "review",
        'url':url,
        'time':format_time,
        'results': comment.get_text()
    })
    
driver.quit()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF748E0FB05+28789]
	(No symbol) [0x00007FF748D786E0]
	(No symbol) [0x00007FF748C1592A]
	(No symbol) [0x00007FF748BEF505]
	(No symbol) [0x00007FF748C96477]
	(No symbol) [0x00007FF748CAEF42]
	(No symbol) [0x00007FF748C8F1E3]
	(No symbol) [0x00007FF748C5A938]
	(No symbol) [0x00007FF748C5BAA1]
	GetHandleVerifier [0x00007FF74914933D+3410093]
	GetHandleVerifier [0x00007FF74915E7DD+3497293]
	GetHandleVerifier [0x00007FF749152A73+3448803]
	GetHandleVerifier [0x00007FF748ED7BBB+848171]
	(No symbol) [0x00007FF748D83C3F]
	(No symbol) [0x00007FF748D7F6E4]
	(No symbol) [0x00007FF748D7F87D]
	(No symbol) [0x00007FF748D6ED49]
	BaseThreadInitThunk [0x00007FFCCBD1259D+29]
	RtlUserThreadStart [0x00007FFCCC4EAF38+40]


In [ ]:
# 將結果轉為data frame
df = pd.DataFrame(listData)
# 取得文章數量
print(len(df))
# 輸出為csv檔
df.to_csv('klook', index=True, encoding='utf-8-sig')